In [5]:
pip install duckdb

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ------------- -------------------------- 4.2/12.3 MB 18.0 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 35.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [44]:
import duckdb

# ① 主连接用内存库（可写），用于放临时对象/设置
con = duckdb.connect()  # 等价于 :memory:

# ② 让 MinIO 可用：httpfs + S3 参数
con.execute("INSTALL httpfs;")
con.execute("LOAD httpfs;")
con.execute("SET s3_endpoint='127.0.0.1:9000';")
con.execute("SET s3_access_key_id='minioadmin';")
con.execute("SET s3_secret_access_key='minioadmin';")
con.execute("SET s3_use_ssl=false;")

# ⭐ MinIO 常用：强制 path-style（否则经常 list-type=2/连接错误）
con.execute("SET s3_url_style='path';")
con.execute("SET s3_region='us-east-1';")

# ③ 关键：把你的 catalog.duckdb 以 localmemdb 名字 attach（只读）
con.execute(r"ATTACH 'C:\Users\mu\Desktop\the-nomad-data-stack\.duckdb\catalog.duckdb' AS catalog (READ_ONLY);")
con.execute("USE catalog;") 

In [66]:
con.execute("SELECT * FROM duckdb_schemas();").df()


,oid,database_name,database_oid,schema_name,comment,tags,internal,sql
0,1999,catalog,2090,ads,None,{},False,None
1,2001,catalog,2090,catalog_meta,None,{},False,None
2,2009,catalog,2090,dim,None,{},False,None
3,2011,catalog,2090,dwd,None,{},False,None
4,1997,catalog,2090,main,None,{},True,None
5,2014,catalog,2090,ods,None,{},False,None
6,2088,catalog,2090,raw,None,{},False,None
7,590,memory,592,main,None,{},True,None
8,2093,system,0,information_schema,None,{},True,None
9,0,system,0,main,None,{},True,None


In [86]:
con.execute("SHOW ALL TABLES;").df()


,database,schema,name,column_names,column_types,temporary
0,catalog,catalog_meta,schema_migrations,"[filename, checksum, applied_at]","[VARCHAR, VARCHAR, TIMESTAMP]",False
1,catalog,ods,ods_etf_universe_whitelist,"[symbol, name, enabled, trade_date, dt]","[VARCHAR, VARCHAR, INTEGER, DATE, VARCHAR]",False
2,catalog,ods,ods_fund_etf_spot_em_akshare,"[symbol, name, amount, trade_date, dt]","[VARCHAR, VARCHAR, DOUBLE, DATE, VARCHAR]",False
3,catalog,ods,ods_fund_names_em_akshare,"[symbol, name, fund_type, pinyin_full, pinyin_...","[VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, ...",False
4,catalog,ods,ods_tool_trade_date_hist_sina_akshare,"[trade_date, dt]","[DATE, VARCHAR]",False


In [98]:
con.execute("select * from ods.ods_fund_etf_spot_em_akshare limit 100").df()

ConnectionException: Connection Error: Connection already closed!

In [96]:
con.close()